In [ ]:
from db import databaseconnection
import numpy as np
from ga import networkedgeneticalgorithm as nga
from membranesimulation import MembraneSimulation
import parlammps
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import networkx
from tools import vectools
import os
import tools
import copy
import random
import time
%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
dbconn = databaseconnection.DatabaseConnection('db/datastore.db')

In [ ]:
dbconn.whatSessions()

In [ ]:
data = dbconn.loadSession('2018-02-18 08:56:07')

In [ ]:
dbconn.close()

### Helper Fns.

In [ ]:
#fitness given genome
def genomeFitness(genome):
    for individual in data['individuals']:
        if np.array_equal(individual['genome'], genome):
            return individual['fitness']
    return -1

In [ ]:
def trimLigands(inds):
    #trim ligands with eps=0 for tiled NP model
    inds_tmp = []
    for ind in inds:
        ligands_tmp = []
        for ligand in ind['phenome'].particle.ligands:
            if ligand.eps != 0:
                ligands_tmp.append(ligand)
        ind_tmp = copy.deepcopy(ind)
        ind_tmp['phenome'].particle.ligands = ligands_tmp
        inds_tmp.append(ind_tmp)
    return inds_tmp

In [ ]:
def evaluateSample(nps, gens, inds, rs, RUNTIME=25000, TIMESTEP=0.01, wd_out = 'wetlab/out', wd_run = 'wetlab/run',nprocs=2, timeout=999999, silent=False, DUMP=1000):    
    wd = ''
    for np,gen,ind in zip(nps,gens,inds):
        for rs_i in range(rs):
            simName = str(gen) + '_' + str(ind) + '_' +str(rs_i)
            sim = MembraneSimulation(
                'sim_'+simName,
                np.particle,
                RUNTIME,
                TIMESTEP,                 
                os.path.join(wd, wd_out),
                os.path.join(wd, wd_run),
                os.path.join(wd,'mem/template/data.template'),
                os.path.join(wd,'mem/template/in.template'),
                rAxis=vectools.randomUnitVector(),
                rAmount=random.uniform(1.571,3.141),
                dumpres=DUMP
                )
            sim.saveFiles()
            scriptPath=os.path.join(sim.filedir,sim.scriptName)
            outFilePath = os.path.join(sim.outdir,sim.outName)
            parlammps.runSim(scriptPath,nprocs,timeout,silent)                        
            sim.postProcessOutput(outFilePath)                        

In [ ]:
def greatArcDist(Ang1, Ang2, rad=4):
    #Ang = (PolarAng,AziAng)
    #https://math.stackexchange.com/questions/231221/great-arc-distance-between-two-points-on-a-unit-sphere
    arcDist=rad*(np.arccos((np.cos(Ang1[0])*np.cos(Ang2[0]))+((np.sin(Ang1[0])*np.sin(Ang2[0]))*(np.cos(Ang1[1]-Ang2[1])))))
    return arcDist

In [ ]:
def cluster(ligands, trimmingDist, silent=True):
    if not silent:
        startTime = time.time()
    #print(greatArcDist((0,0), ((3.14/15.0),0))) #bad at small distances, correct should be 0.8377 but returns 0.8373, willing to accept +- 0.001

    ligandsTmp = copy.deepcopy(ligands)

    clusters = []
    while ligandsTmp:    
        #print('progress: {}/{}'.format(len(ligandsTmp),len(ligands)))
        nextSeedQueue = [ligandsTmp[0]]    

        clusterTmp = []
        clusterTmp.append(ligandsTmp[0])
        del ligandsTmp[0]
        while nextSeedQueue:
            seed = nextSeedQueue.pop()
            for ligand in ligandsTmp:
                if greatArcDist((seed.polAng, seed.aziAng),(ligand.polAng,ligand.aziAng)) <= trimmingDist:                
                    nextSeedQueue.append(ligand)
                    clusterTmp.append(ligand)
                    ligandsTmp.remove(ligand)
        clusters.append(clusterTmp)                  
    if not silent:
        print('ligands: {}'.format(len(ligands)))
        print('clusters: {}'.format(len(clusters)))
        print('clustered in {}s'.format(time.time()-startTime))
    return clusters

In [ ]:
def avgClusterDist(clusters, silent=True):
    ###find average position of each cluster
    if not silent:
        startTime = time.time()
    avgClusterPos = []    
    for i in clusters:
        ligandCount = 0
        totalPol = 0
        totalAzi = 0
        for j in i:
            totalPol += j.polAng
            totalAzi += j.aziAng
            ligandCount += 1.0
        avgClusterPos.append(((totalPol/ligandCount),(totalAzi/ligandCount)))
    ###
    ###find distances to next nearest for each
    avgClusterNxtDistances = []
    for i in avgClusterPos:
        tmp = []
        for j in avgClusterPos:
            if i != j:
                tmp.append(greatArcDist(i,j))
        avgClusterNxtDistances.append(np.min(tmp))
#     avgClusterNxtDistances = []    
#     avgClusterPosTmp = copy.deepcopy(avgClusterPos)
#     while avgClusterPosTmp:
#         cClusterPos = avgClusterPosTmp.pop()        
#         if len(avgClusterPosTmp) > 0:
#             tmp = []
#             for i in avgClusterPosTmp:
#                 if i != cClusterPos:
#                     tmp.append(greatArcDist(i,cClusterPos))
#             avgClusterNxtDistances.append(np.min(tmp))
    ###
    ###return an average of those distances   
    if not silent:
        print('avg cluster distance found in {}s'.format(time.time() - startTime))    
    return sum(avgClusterNxtDistances)/len(avgClusterNxtDistances)
    ###

In [ ]:
def avgLigandDist(ligands, silent=True):
    if not silent:
        startTime = time.time()
    LigandNxtDistances = []
    for i in ligands:
        tmp = []
        for j in ligands:
            if i != j:
                tmp.append(greatArcDist((i.polAng,i.aziAng),(j.polAng,j.aziAng)))
        LigandNxtDistances.append(np.min(tmp))
#     ligandsTmp = copy.deepcopy(ligands)
#     while ligandsTmp:
#         cLigand = ligandsTmp.pop()
#         if len(ligandsTmp) > 0:
#             tmp = []
#             for i in ligandsTmp:
#                 if i != cLigand:
#                     tmp.append(greatArcDist((i.polAng,i.aziAng),(cLigand.polAng,cLigand.aziAng)))
#             LigandNxtDistances.append(np.min(tmp))
    if not silent:
        print('avg ligand distance found in {}s'.format(time.time() - startTime))
    return sum(LigandNxtDistances)/len(LigandNxtDistances)

In [ ]:
def numNN(ligands, cutoff):
    #number of ligands that are nearest neighbours ligand-wise
    numNNs = []
    for i in ligands:
        numNN_i = 0
        for j in ligands:
            if i != j:
                if greatArcDist((i.polAng,i.aziAng),(j.polAng,j.aziAng)) <= cutoff:
                    numNN_i += 1
        numNNs.append(numNN_i)
    return numNNs

In [ ]:
def numMinNumNN(ligands,cutoff,minligands):
    #number of ligands which have at least minligands ligands that are within cutoff range ligand-wise
    numMinNumNNs = 0
    for i in ligands:
        numNNs = 0
        for j in ligands:
            if i != j:
                if greatArcDist((i.polAng,i.aziAng),(j.polAng,j.aziAng)) <= cutoff:
                    numNNs += 1
        if numNNs >= minligands:
            numMinNumNNs += 1
    return numMinNumNNs

In [ ]:
def totalEps(ligands):
    return np.sum([ligand.eps for ligand in ligands])    

In [ ]:
def plotNP(ligands=None, rad=4):
    fig = plt.figure(figsize=(40.0,16.0))
    ax = fig.gca(projection='3d')
    ax.set_aspect("equal")

    u, v = np.mgrid[0:2*np.pi:25j, 0:np.pi:25j]
    x = np.cos(u)*np.sin(v)*rad
    y = np.sin(u)*np.sin(v)*rad
    z = np.cos(v)*rad
    #ax.scatter(x, y, z, color="r", alpha=0.2)    
    ax.plot_wireframe(x, y, z, color="r", alpha=0.2)
    if ligands:
        for ligand in ligands:
            v = tools.vectools.polarToCartesianVector(rad,ligand.polAng,ligand.aziAng)
            ax.scatter(v[0],v[1],v[2],s=[200]*len(ligands),color="g")

# GA Metrics

In [ ]:
#print(data['metrics'])
plt.rcParams['figure.figsize'] = (10.0, 8.0)
fitAvg = [metric['avg'] for metric in data['metrics']]
fitMin = [metric['min'] for metric in data['metrics']]
fitMax = [metric['max'] for metric in data['metrics']]
plt.plot(fitAvg, '.',label='Average Fitness');
plt.plot(fitMax, '.',label='Max Fitness');
plt.plot(fitMin, '.',label='Min Fitness');
plt.xlabel('generation number')
plt.ylabel('fitness')
plt.legend();

# Genealogy

In [ ]:
plt.rcParams['figure.figsize'] = (40.0, 16.0)
graph = networkx.DiGraph(data['genealogy']['tree'])
graph = graph.reverse()
colors = [genomeFitness(data['genealogy']['history'][i]) for i in graph]
positions = networkx.drawing.nx_agraph.graphviz_layout(graph, prog="dot")
networkx.draw(graph, positions, node_color = colors, s=40, cmap=plt.cm.Spectral)
fits = [ individual['fitness'] for individual in data['individuals']]
sm = plt.cm.ScalarMappable(cmap=plt.cm.Spectral, norm=plt.Normalize(vmin=float(min(fits)), vmax=float(max(fits))))
sm._A = []
plt.colorbar(sm)
plt.savefig('genealogy.png')
plt.show()

In [ ]:
# print(data['metrics'])
# print(len(data['genealogy']['tree']))
# print(len(data['genealogy']['history']))
# print(len(data['individuals']))
# print(data['genealogy']['tree'][1])
# print(data['genealogy']['history'][1])
#print(data['individuals'][0]['genome'])
#print(data['individuals'][0]['fitness'])
#print(genomeFitness(data['individuals'][0]['genome']))
print(len(data['individuals']))

# Generate Local Samples

In [ ]:
#generate your samples
hof = []
for i in data['individuals']:
    if i['fitness'] = 400.0: #400 means all rotations budded in!
        hof.append(i)

# #best performers in gen X (gen=X)
# curr_best_fitness = -1
# best = None
# genS = 29
# for i in data['individuals']:
#     if i['gen'] == genS:
#         if i['fitness'] > curr_best_fitness:    
#             best = i
#             curr_best_fitness = i['fitness']
# for i in data['individuals']:
#     if i['gen'] == genS:
#         if i ['fitness'] == curr_best_fitness:            
#             hof.append(i)
# #hof.append(best)        
    
print('samples length:{}'.format(len(hof)))
print('samples generations:{}'.format([i['gen'] for i in hof]))

In [ ]:
phenomes = [i['phenome'] for i in hof]
gens = [i['gen'] for i in hof]
inds = [i['ind'] for i in hof]
evaluateSample(phenomes, gens, inds,rs=2,nprocs=4,silent=True)
#evaluateSample(phenomes, gens, inds,rs=2,nprocs=4,silent=False,DUMP=250)
#evaluateSample(phenomes, gens, inds,rs=1,silent=True,RUNTIME=100)

# Ligand Clustering

In [ ]:
# clusters = cluster(hof[0]['phenome'].particle.ligands,1.8,silent=False)
# plotNP(clusters[9])

In [ ]:
#NNcutoff = 1.345 #for int. range 1.8
NNcutoff = 1.559 #for int. range 2.0
for i in data['individuals']:    
    i['ligand_clusters'] = cluster(i['phenome'].particle.ligands,NNcutoff,silent=True)

In [ ]:
##for tiled NP, removed ligands with eps=0
##data['individuals'] = trimLigands(data['individuals'])

## v. Gen